### **Pydantic**
#### ***Assignment-1***

Create a simple assistant that uses any LLM and should be Pydantic, When we ask about any product, It should give you two information. `"Product Name"` and `"Prodcut Details"` along with tentative price(integer) in `"USD"`. Use Both `"Chat Prompt Template"` and `"Prompt Teamplate"`.

In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser, StrOutputParser
from pydantic import BaseModel, Field 

In [2]:
model= ChatGoogleGenerativeAI(model="gemini-1.5-flash")
response= model.invoke("Hi")
print(response.content)

Hi there! How can I help you today?


In [3]:
class Product(BaseModel):
    product_name: str = Field(description="Product Name")
    product_details: str = Field(description="Prodcut's detail")

In [4]:
parser= PydanticOutputParser(pydantic_object=Product)

In [5]:
output_parser= parser.get_format_instructions()

In [32]:
output_parser

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"product_name": {"description": "Product Name", "title": "Product Name", "type": "string"}, "product_details": {"description": "Prodcut\'s detail", "title": "Product Details", "type": "string"}}, "required": ["product_name", "product_details"]}\n```'

In [6]:
prompt= PromptTemplate(
    template="""You are a smart AI assistant, {format_instructions}\n Based on user's input provide details along with tentative price in USD
    
    user_input {question}""",
    input_variables=["question"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

chain= prompt | model | parser
response= chain.invoke({"question": "Samsung Mobile"})
print(response)

product_name='Samsung Mobile' product_details='This refers to a wide range of Samsung mobile phones.  To give you a more accurate price, please specify the model (e.g., Galaxy S23, Galaxy A54, Galaxy Z Fold4). Prices vary greatly depending on the model, storage capacity, and retailer.  Tentative prices range from around $200 for budget models to over $1000 for flagship devices.'


In [11]:
response= chain.invoke({"question": "iPhone Mobile"})

In [14]:
print(response)

product_name='iPhone Mobile' product_details="Apple iPhones are a popular series of smartphones known for their sleek design, powerful performance, and high-quality cameras.  Specific details depend on the model (e.g., iPhone 14, iPhone 13, iPhone SE). Features often include a large, vibrant display, advanced processors, long battery life, and access to a vast app ecosystem.  They typically run on Apple's iOS operating system."


In [ ]:
prompt= ChatPromptTemplate(
    [
        ("ai", "You are a smart assistant, {format_instructions}\nBased on user's input provide details along with tentative price in USD\n user_input {question}"),
        ("human", "{question}"),
    ]
)

chain= prompt | model
response= chain.invoke({
    "format_instructions": parser.get_format_instructions(),
    "question": "Samsung Mobile"
    }
)
print(response.content)

```json
{
  "product_name": "Samsung Galaxy S23 Ultra",
  "product_details": "The Samsung Galaxy S23 Ultra boasts a stunning 6.8-inch Dynamic AMOLED 2X display with a 120Hz adaptive refresh rate.  It's powered by a Snapdragon 8 Gen 2 processor and offers a versatile camera system, including a 200MP main sensor, a 12MP ultrawide lens, and two 10MP telephoto lenses (one with 3x optical zoom and one with 10x optical zoom).  It also features a built-in S Pen for note-taking and drawing, and a long-lasting battery with fast charging capabilities. Available in various colors and storage options."
}
```


In [16]:
response= chain.invoke({
    "format_instructions": parser.get_format_instructions(),
    "question": "iPhone Mobile"
    }
)
print(response.content)

```json
{
  "product_name": "iPhone 14 Pro Max",
  "product_details": "The iPhone 14 Pro Max boasts a stunning 6.7-inch Super Retina XDR display with ProMotion technology for smooth scrolling and a responsive touch.  It's powered by the A16 Bionic chip, delivering exceptional performance and efficiency.  The advanced camera system features a 48MP Main camera, a 12MP Ultra Wide camera, and a 12MP Telephoto camera, allowing you to capture breathtaking photos and videos.  Other features include a durable Ceramic Shield front cover, long battery life, and Emergency SOS via satellite."
}
```
